In [1]:
from rrgcn import RRGCNEmbedder
from torch_geometric.datasets import Entities
import torch
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

ModuleNotFoundError: No module named 'rrgcn'

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
dataset = Entities('./', 'aifb')
data = dataset[0].to(device)

In [40]:
embedder = RRGCNEmbedder(num_nodes=data.num_nodes, num_relations=dataset.num_relations, num_layers=2, emb_size=2048, device=device)

In [41]:
embs = embedder.embeddings(data.edge_index, data.edge_type, idx=torch.hstack((data.train_idx, data.test_idx)))

100%|██████████| 1/1 [00:01<00:00,  1.82s/it]


In [42]:
train_embs, val_embs, y_train, y_val = train_test_split(embs[:len(data.train_idx)], data.train_y, test_size=0.1, random_state=42)
test_embs = embs[len(data.train_idx):]

In [43]:
task_type = 'GPU' if torch.cuda.is_available() else 'CPU'
clf = CatBoostClassifier(iterations=1000, early_stopping_rounds=10, task_type=task_type, random_seed=42)
clf = clf.fit(train_embs.cpu().numpy(), y_train.cpu().numpy(), eval_set=(val_embs.cpu().numpy(), y_val.cpu().numpy()))

Learning rate set to 0.085304
0:	learn: 1.3108780	test: 1.3386966	best: 1.3386966 (0)	total: 36.9ms	remaining: 36.9s
1:	learn: 1.2528786	test: 1.2955216	best: 1.2955216 (1)	total: 58.8ms	remaining: 29.4s
2:	learn: 1.2042893	test: 1.2689032	best: 1.2689032 (2)	total: 82.6ms	remaining: 27.5s
3:	learn: 1.1538607	test: 1.2444230	best: 1.2444230 (3)	total: 107ms	remaining: 26.6s
4:	learn: 1.0782380	test: 1.1890538	best: 1.1890538 (4)	total: 129ms	remaining: 25.8s
5:	learn: 1.0342293	test: 1.1670562	best: 1.1670562 (5)	total: 152ms	remaining: 25.3s
6:	learn: 0.9920668	test: 1.1366792	best: 1.1366792 (6)	total: 176ms	remaining: 24.9s
7:	learn: 0.9386813	test: 1.1121528	best: 1.1121528 (7)	total: 198ms	remaining: 24.5s
8:	learn: 0.9018029	test: 1.0776206	best: 1.0776206 (8)	total: 221ms	remaining: 24.3s
9:	learn: 0.8689281	test: 1.0686357	best: 1.0686357 (9)	total: 243ms	remaining: 24.1s
10:	learn: 0.8385524	test: 1.0610842	best: 1.0610842 (10)	total: 267ms	remaining: 24s
11:	learn: 0.8143026	

In [44]:
print(classification_report(clf.predict(test_embs.cpu().numpy()), data.test_y.cpu().numpy()))

              precision    recall  f1-score   support

           0       1.00      0.79      0.88        19
           1       0.67      1.00      0.80         2
           2       0.67      1.00      0.80         4
           3       0.83      0.91      0.87        11

    accuracy                           0.86        36
   macro avg       0.79      0.92      0.84        36
weighted avg       0.89      0.86      0.86        36

